In [1]:
# python.exe -m pip install --upgrade pip
# pip install ipykernel
# pip install jupyter
# python -m pip install bitsandbytes --prefer-binary --extra-index-url=https://jllllll.github.io/bitsandbytes-windows-webui
# pip install langchain einops accelerate transformers scipy xformers==0.0.20
# pip install torch==2.0.1+cu117 torchvision==0.15.2+cu117 -f https://download.pytorch.org/whl/torch_stable.html
# pip install sentencepiece sentence_transformers
# pip install llama-index
# pip install llama-hub
# pip install invisible_watermark safetensors
# pip install diffusers

In [1]:
# Import tranformer classes
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer
from transformers import BitsAndBytesConfig
# Import torch
import torch

In [2]:
torch.cuda.is_available()

True

In [3]:
# model name
name = "meta-llama/Llama-2-7b-chat-hf"
# name = "togethercomputer/LLaMA-2-7B-32K"
# Set auth variable from hugging face
auth_token = ""

In [4]:
# Create tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    name,
    cache_dir="./model/",
    use_auth_token=auth_token,
)

c:\Users\Sagar\Python files_Jupiter\Git Repo Local\GAN Poetry\project\Lib\site-packages\transformers\models\auto\tokenization_auto.py:655: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [5]:
# Create quantization config
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4",
)

In [6]:
# Create model
model = AutoModelForCausalLM.from_pretrained(
    name,
    cache_dir="./model/",
    use_auth_token=auth_token,
    rope_scaling={"type":"dynamic","factor":2},
    load_in_4bit=True,
    quantization_config=quantization_config,
)

c:\Users\Sagar\Python files_Jupiter\Git Repo Local\GAN Poetry\project\Lib\site-packages\transformers\models\auto\auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


bin c:\Users\Sagar\Python files_Jupiter\Git Repo Local\GAN Poetry\project\Lib\site-packages\bitsandbytes\libbitsandbytes_cuda117.dll


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

c:\Users\Sagar\Python files_Jupiter\Git Repo Local\GAN Poetry\project\Lib\site-packages\transformers\utils\hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [7]:
# Setup prompt
prompt = "### User:Generate a poem on nature. ### Assistant:"
# Pass the prompt to the tokenizer
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
# Setup text streamer
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_token=True)

In [8]:
# Actually run the thing
output = model.generate(inputs["input_ids"], streamer=streamer, use_cache=True, max_new_tokens=float("inf"))

Sure! Here is a poem about nature:

In the grand expanse of earth and sky,
A world of wonder, full of life and might.
From mountains high to valleys low,
The beauty of nature, we are told.

The trees stand tall, their leaves a-glow,
A canopy of green, where birds do flow.
The rivers run, with waters clear,
A pathway to the sea, so pure and dear.

The wind it blows, with gentle grace,
And whispers secrets, in a lover's place.
The sun it shines, with radiant light,
Bringing life to all, with its beaming sight.

The flowers bloom, in every hue,
A kaleidoscope of color, pure and true.
The bees they buzz, with busy cheer,

In nature's embrace, we find our peace,
A sense of calm, a world to cease.
From the hustle and bustle of the day,
We find solace, in nature's sway.

So let us cherish, this world so grand,
And do our part, to preserve its land.
For in nature's beauty, we find our home,
And a world of wonder, forever known.</s>


In [9]:
# Convert the output tokens back to text
output_text = tokenizer.decode(output[0], skip_special_tokens=True)

In [10]:
print(output_text)

### User:Generate a poem on nature. ### Assistant: Sure! Here is a poem about nature:

In the grand expanse of earth and sky,
A world of wonder, full of life and might.
From mountains high to valleys low,
The beauty of nature, we are told.

The trees stand tall, their leaves a-glow,
A canopy of green, where birds do flow.
The rivers run, with waters clear,
A pathway to the sea, so pure and dear.

The wind it blows, with gentle grace,
And whispers secrets, in a lover's place.
The sun it shines, with radiant light,
Bringing life to all, with its beaming sight.

The flowers bloom, in every hue,
A kaleidoscope of color, pure and true.
The bees they buzz, with busy cheer,

In nature's embrace, we find our peace,
A sense of calm, a world to cease.
From the hustle and bustle of the day,
We find solace, in nature's sway.

So let us cherish, this world so grand,
And do our part, to preserve its land.
For in nature's beauty, we find our home,
And a world of wonder, forever known.


In [15]:
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
from transformers import pipeline

In [20]:
# Create llm pipeline
pipe = pipeline("text-generation",model=model,tokenizer=tokenizer,max_new_tokens=1024)
llm = HuggingFacePipeline(pipeline=pipe)

In [26]:
# Create prompt template
prompt = PromptTemplate(
    input_variables=["question"],
    template="""<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as 
helpfully as possible, while being safe. Your answers should not include
any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.
Please ensure that your responses are socially unbiased and positive in nature. 

Your goal is to only compose poems and reply in poetic speech. Don't reply if the request is not relate to composing a poem.<</SYS>>

HUMAN: {question}

ASSISTANT:[/INST]""")

In [27]:
# LLMChain
chain = LLMChain(prompt=prompt,llm=llm)

In [28]:
response = chain.run("Write a poem on nature.")

In [29]:
print(response)

  Of course, I'd be delighted to assist you in crafting a poem on nature. Here's a piece of verse that I hope you'll find pleasing:

Nature's beauty, oh so grand,
A symphony of sights, sounds, and scents.
From mountains tall and valleys deep,
To oceans vast and skies so high,
She weaves a tapestry of life,
A kaleidoscope of wonder and delight.

Trees stand tall, their leaves a-glow,
A canopy of green above,
A haven for birds to sing and fly,
A home for creatures to thrive and grow.

Rivers flow, their currents strong,
A path for life to move along,
A source of life and vitality,
A symbol of resilience and grit.

Seas and oceans, vast and wide,
A world of wonder, a world to hide,
A place where creatures roam and play,
A home for life in every way.

The wind it blows, the sun it shines,
A balm for nature's gentle lines,
A reminder of life's grace and might,
A testament to nature's beauty bright.

I hope this poem captures some of the beauty and wonder of nature. Please let me know if the